# Lee un año de datos a ver cómo se hace más rápido

In [7]:
### Probando con datos
import pyspark
from pyspark.sql import SparkSession
from pyspark.conf import SparkConf

conf = (SparkConf().set("spark.executor.memory", "6g")   #<--- Por executor. Como hay dos instancias, la mitad por executor.
                    .set("spark.executor.cores", "3")    #<--- Por cada executor. Como hay 4, 4*3=12 cores se usan.
                    .set("spark.executor.instances", "2")
                    .set("spark.eventLog.enabled", "true"))
spark = SparkSession.builder.master("spark://spark-master:7077").appName("ReadParquetYearMon16MM").config(conf=conf).getOrCreate()
spark

In [8]:
import time
print ('Inicio: '+time.strftime("%c"))
inicio = time.perf_counter()

Inicio: Wed Nov  9 18:10:10 2022


In [28]:
%%time
from pyspark.sql.types import StructType
from pyspark.sql.types import StringType,BooleanType,DateType,IntegerType
#from pyspark.sql.functions import sum, col, desc, when, lit
from pyspark.sql.functions import *


esquema = StructType() \
      .add("Id",StringType(),True) \
      .add("fecNac",StringType(),True) \
      .add("sex",StringType(),True) \
      .add("ingresos",IntegerType(),True) \
      .add("partido",StringType(),True) \
      .add("edad",IntegerType(),True) \
      .add("mapa",StringType(),True) \
      .add("nombre",StringType(),True) \
      .add("lider",StringType(),True) \
      .add("fechaCreacion",StringType(),True)

"""
 * Bucle año/mes
 
 1.- Lee un año mes a mes: lo hace como a menos de un segundo por mes.
 2.- Le pongo 10 años
"""
total = 0

anomes = '1931-01'

print(132*'='+'\nTratando partición yearMon='+anomes)
inicioP = time.perf_counter()

df = spark.read.option("mergeSchema", True).option("schema", "esquema").option("basePath", "file:///opt/workspace/datos/fakePeopleConPartido.parquet/")\
               .format("parquet").load(['./datos/fakePeopleConPartido.parquet/yearMon='+anomes])

#df.printSchema()

# Cast --> le cambia el formato a una columna:

df1=df.withColumn('edad',col=col('edad').cast('double'))

#df1.printSchema()

"""
 * El when--otherwise se puede escribir de dos formas:
"""

df2=df1.withColumn('calificacion',when(col('partido')=='VOX',lit('sensato'))
                                 .when(col('partido')=='PP',lit('bobo'))
                                 .otherwise(lit('idiota')))
                   
df2.show()

df3=df1.withColumn('calificacion',when(df1.partido =='VOX',lit('sensato'))
                                 .when(df1.partido =='PP',lit('bobo'))
                                 .otherwise(lit('idiota')))

df3.show()

#Cuenta cuántos nan or null hay en cada columna:

print(df.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df.columns]).show())


Tratando partición yearMon=1931-01
+------------+----------+---+--------+-------+----+-------------+--------------------+--------------------+-------------+-------+------------+
|          Id|    fecNac|sex|ingresos|partido|edad|         mapa|              nombre|               lider|fechaCreacion|yearMon|calificacion|
+------------+----------+---+--------+-------+----+-------------+--------------------+--------------------+-------------+-------+------------+
|ID1151795346|1931-01-11|  M|   13400|    VOX|91.0|1-1-0-1-0-1-1|     VOX España S.A.|    Santiago Abascal|   2015-01-01|1931-01|     sensato|
|ID1145148063|1931-01-12|  M|   22100|     PP|91.0|0-1-1-1-1-1-1|     Partido Popular|Alberto Núñez Feijóo|   1986-10-31|1931-01|        bobo|
|ID1145149777|1931-01-04|  H|   36600|     CS|91.0|1-0-0-0-1-1-1|          Ciudadanos|      Inés Arrimadas|   2008-04-18|1931-01|      idiota|
|ID1142838013|1931-01-09|  H|   25400|     UP|91.0|0-0-0-0-0-1-1|      Unidas Podemos|        Yolanda Díaz|

+---+------+---+--------+-------+----+----+------+-----+-------------+-------+
| Id|fecNac|sex|ingresos|partido|edad|mapa|nombre|lider|fechaCreacion|yearMon|
+---+------+---+--------+-------+----+----+------+-----+-------------+-------+
|  0|     0|  0|       0|      0|   0|   0|     0|    0|            0|      0|
+---+------+---+--------+-------+----+----+------+-----+-------------+-------+

None
CPU times: user 15.6 ms, sys: 11 ms, total: 26.5 ms
Wall time: 2.26 s


In [ ]:
ct = df.count()
print(ct)
total = total + ct
print ('Final partición: '+time.strftime("%c"))
final = time.perf_counter()
print(f'Hecho en {round(final - inicioP, 4)} segundo(s)')


In [6]:
help(df.withColumn)

Help on method withColumn in module pyspark.sql.dataframe:

withColumn(colName, col) method of pyspark.sql.dataframe.DataFrame instance
    Returns a new :class:`DataFrame` by adding a column or replacing the
    existing column that has the same name.
    
    The column expression must be an expression over this :class:`DataFrame`; attempting to add
    a column from some other :class:`DataFrame` will raise an error.
    
    .. versionadded:: 1.3.0
    
    Parameters
    ----------
    colName : str
        string, name of the new column.
    col : :class:`Column`
        a :class:`Column` expression for the new column.
    
    Notes
    -----
    This method introduces a projection internally. Therefore, calling it multiple
    times, for instance, via loops in order to add multiple columns can generate big
    plans which can cause performance issues and even `StackOverflowException`.
    To avoid this, use :func:`select` with the multiple columns at once.
    
    Examples
    

In [4]:
print ('Final: '+time.strftime("%c"))
final = time.perf_counter()
print(f'Hecho en {round(final - inicio, 4)} segundo(s)')

print('TOTAL de registros leidos '+str(total))

Final: Wed Nov  9 18:05:20 2022
Hecho en 5.5997 segundo(s)
TOTAL de registros leidos 802937


In [5]:
spark.sparkContext.stop()
print('Sacabao')

Sacabao
